In [1]:
import json
import re
import spacy as sp
import pandas as pd

with open('data/all_situations.json') as f:
    situations = json.load(f)

print(f'There are {len(situations)} situations in the dataset')

There are 1574 situations in the dataset


## Exploratory Data Analysis

#### Create dataframe with useful information

In [2]:
situation_data = {
    'id': [situation['id'] for situation in situations],
    'situation_title': [situation['title'] for situation in situations],
    'related_subject': [situation['subject'].get('title', None) if situation['subject'] else None for situation in situations],
    'content': [situation['content'] for situation in situations],
    'icpc': [', '.join(situation['codes']['icpc']) if situation['codes']['icpc'] else None for situation in situations],
    'medical_source': [', '.join(situation['codes']['medical_source']) if situation['codes']['medical_source'] else None for situation in situations],
    'related_situations': [', '.join(related_situation['title'] for related_situation in situation['related']['situations']) if situation['related']['situations'] else None for situation in situations]
}

situation_df = pd.DataFrame(situation_data)

situation_df

,id,situation_title,related_subject,content,icpc,medical_source,related_situations
0,16574,Ik ga naar de oogarts voor een oogontsteking d...,Oogontsteking door herpes,"[{'index': 0, 'type': 'text', 'category': 'In ...",F73,FMS,"Ik heb een oogontsteking door herpes, De oogon..."
1,11567,Ik moet soms ineens heel erg plassen en ben da...,Plas niet goed kunnen ophouden bij vrouwen,"[{'index': 0, 'type': 'text', 'category': 'In ...",U04,NHG,"Ik kan mijn plas niet goed ophouden (vrouw), I..."
2,11568,Ik kan mijn plas niet goed ophouden bij bijvoo...,Plas niet goed kunnen ophouden bij vrouwen,"[{'index': 0, 'type': 'text', 'category': 'In ...",U04,NHG,"Ik kan mijn plas niet goed ophouden (vrouw), I..."
3,12184,Ik ga mijn blaas trainen om mijn plas langer o...,Overactieve blaas,"[{'index': 0, 'type': 'text', 'category': 'In ...","u01, U04",NHG,"Ik heb een overactieve blaas, Ik ga bekkenbode..."
4,12148,Ik heb een overactieve blaas,Overactieve blaas,"[{'index': 0, 'type': 'text', 'category': 'In ...","u02, U04",NHG,Ik ga mijn blaas trainen om mijn plas langer o...
...,...,...,...,...,...,...,...
1569,12295,Ik wil mijn brandwond behandelen,Brandwonden,"[{'index': 0, 'type': 'text', 'category': 'In ...",S14,None,"Hoe kan ik verbranding voorkomen?, Mijn kind h..."
1570,11306,Ik heb me verbrand,Brandwonden,"[{'index': 0, 'type': 'text', 'category': 'In ...",S14,None,"Hoe kan ik verbranding voorkomen?, Ik wil mijn..."
1571,17604,Ik heb een plekje op mijn voorhuid,Plekje op de voorhuid,"[{'index': 0, 'type': 'text', 'category': 'In ...",Y04,FMS,None
1572,15563,Mijn baby is ziek geworden van de GBS bacterie...,Zwanger en de GBS bacterie,"[{'index': 0, 'type': 'text', 'category': 'In ...",None,FMS,Ik heb verhoogde kans dat mijn baby ziek wordt...


#### Exploration of ICPC codes

In [3]:
situations_no_icpc = []
situations_icpc = []

for i in range(len(situation_df)):
    situation_id = situation_df['id'][i]
    situation_title = situation_df['situation_title'][i]
    icpc_code = situation_df['icpc'][i]

    if icpc_code is None:
        situations_no_icpc.append((situation_id, situation_title))
    else:
        # Check for invalid ICPC codes and correct them
        if ';' in icpc_code:
            icpc_code = icpc_code.replace(';', ',')
            situation_df.loc[situation_df['id'] == situation_id, 'icpc'] = icpc_code

        situations_icpc.append((situation_id, situation_title, icpc_code))

print(f'Number of situations with no ICPC codes: {len(situations_no_icpc)}')
print(f'Number of situations with ICPC codes: {len(situations_icpc)}')

Number of situations with no ICPC codes: 258
Number of situations with ICPC codes: 1316


In [4]:
situation_multiple_icpc = [item for item in situations_icpc if ',' in item[2]]
print(f'Number of situations with multiple ICPC codes: {len(situation_multiple_icpc)}')

Number of situations with multiple ICPC codes: 401


In [5]:
situation_specific_icpc = [item for item in situations_icpc if '.' in item[2]]
print(f'Number of situations with specific ICPC codes: {len(situation_specific_icpc)}')

Number of situations with specific ICPC codes: 78


#### Exploration of medical sources

In [6]:
situations_no_medical_source = []
for i in range(len(situation_df)):
    if situation_df['medical_source'][i] is None:
        situations_no_medical_source.append((situation_df['id'][i], situation_df['situation_title'][i]))

print(f'Number of situations with no medical source: {len(situations_no_medical_source)}')
situations_no_medical_source

Number of situations with no medical source: 28


[(12315, 'Hoe kan ik verbranding voorkomen?'),
 (11297, 'Ik ben uitgenodigd voor het bevolkingsonderzoek borstkanker'),
 (12233, 'Ik maak me zorgen om iemand met psychische klachten'),
 (11906, 'Ik word onderzocht op epilepsie'),
 (11905, 'Ik heb een aanval gehad'),
 (11907, 'Ik wil goed omgaan met mijn epilepsie'),
 (15165,
  'Ik ben zwanger en wil mijn baby beschermen tegen kinkhoest (22 wekenprik)'),
 (11268, 'Ik heb hoge bloeddruk'),
 (11824, 'Ik wil mijn botten sterk houden en voorkomen dat ik een bot breek'),
 (11956, 'Ik heb klachten door de eikenprocessierups'),
 (11257, 'Ik heb astma'),
 (11284, 'Ik gebruik medicijnen voor astma'),
 (19423,
  'Ik heb een uitnodiging gekregen voor mijn kind van 14 maanden voor prikken'),
 (14496, 'Ik ga medicijnen gebruiken bij een hoog cholesterol'),
 (11141, 'Ik geef over'),
 (12279, 'Ik heb een ontsteking van de huid rond mijn mond'),
 (12422, 'Ik heb een keratoacanthoom'),
 (21072, 'Ik heb een plekje op mijn huid en wil weten wat het is'),


In [7]:
for situation in situations_no_medical_source:
    meer_informatie = situation_df[situation_df['id'] == situation[0]]['content'].values[0][-1:]
    for item in meer_informatie:
        item_value = item['item'][0]['value']  
        if 'richtlijn' in item_value:  # Check if the word "richtlijn" is present in item_value
            print("The word 'richtlijn' is present in the value:", situation)
        else:
            print(f"The word 'richtlijn' is not present in the value:", situation)

The word 'richtlijn' is present in the value: (12315, 'Hoe kan ik verbranding voorkomen?')
The word 'richtlijn' is present in the value: (11297, 'Ik ben uitgenodigd voor het bevolkingsonderzoek borstkanker')
The word 'richtlijn' is not present in the value: (12233, 'Ik maak me zorgen om iemand met psychische klachten')
The word 'richtlijn' is not present in the value: (11906, 'Ik word onderzocht op epilepsie')
The word 'richtlijn' is not present in the value: (11905, 'Ik heb een aanval gehad')
The word 'richtlijn' is not present in the value: (11907, 'Ik wil goed omgaan met mijn epilepsie')
The word 'richtlijn' is present in the value: (15165, 'Ik ben zwanger en wil mijn baby beschermen tegen kinkhoest (22 wekenprik)')
The word 'richtlijn' is present in the value: (11268, 'Ik heb hoge bloeddruk')
The word 'richtlijn' is present in the value: (11824, 'Ik wil mijn botten sterk houden en voorkomen dat ik een bot breek')
The word 'richtlijn' is not present in the value: (11956, 'Ik heb kla

#### Exploration of overarching subjects

In [8]:
situations_no_subject = []
for i in range(len(situation_df)):
    if situation_df['related_subject'][i] is None:
        situations_no_subject.append((situation_df['id'][i], situation_df['situation_title'][i]))

print(f'Number of situations with no overarching subject: {len(situations_no_subject)}')
situations_no_subject

Number of situations with no overarching subject: 11


[(11329, 'Ik denk na over de griepprik'),
 (27397, 'Mijn partner heeft de ziekte van Parkinson'),
 (27605, 'Ik wil leren omgaan met de ziekte van Parkinson'),
 (27391, 'Ik ben bang dat ik de ziekte van Parkinson heb'),
 (22343, 'Ik word getest op griep'),
 (22357, 'Ik heb griep en ik krijg virusremmers'),
 (11328, 'Ik heb griep'),
 (12453, 'Ik kies voor een maagverkleining'),
 (12454, 'Ik heb een maagverkleining gehad'),
 (22367, 'Ik heb griep en ik word opgenomen in het ziekenhuis'),
 (18320, 'Ik denk na over een maagverkleining en ik wil graag een kind')]

#### Exploration of categories in content

In [10]:
categories = {}

for situation in situations:
    for content_item in situation.get('content', []):
        content_category = content_item.get('category')
        categories[content_category] = categories.get(content_category, 0) + 1

print("Categories and their counts:")
for category, count in categories.items():
    print(f"{category}: {count}")

print(f'There are {len(categories)} categories in the data.')

Categories and their counts:
In het kort: 1573
Wat is het: 995
Onderzoeken: 291
Behandeling: 418
Adviezen: 733
Hoe gaat het verder: 990
Wanneer bellen: 833
Meer informatie: 1535
Hoe werkt plas ophouden: 2
Oorzaken: 596
Hulpmiddelen: 10
Medicijnen: 325
Zijn er medicijnen?: 1
Hoe doe ik het: 2
Plasdagboek: 1
Hulp bij blaastraining: 1
Plas ophouden: 1
Blaastraining: 2
Wat is het?: 25
Geen wondroos krijgen: 1
Wat kan helpen?: 1
Goed voor je huid zorgen: 1
Wondjes verzorgen: 1
Over deze tekst: 65
Film Wondroos: 2
Wat merk ik: 462
Wie heeft grotere kans: 1
Kan het kwaad: 116
Adviezen niet weer wondroos: 1
Wat kan ik zelf doen: 2
Film: 168
Besmetting: 4
Prik: 3
Wanneer: 18
Wanneer niet: 4
Voordelen en nadelen: 24
Griepprik: voor wie: 1
Film: Uitnodiging voor de griepprik: 1
Griepprik: wanneer: 1
Griepprik: werking: 1
Griepprik: elk jaar: 1
Griepprik: voordelen en nadelen: 1
Wat is griep: 1
Film: Griep voorkomen: 1
Besmetting voorkomen: 1
Prik tegen pneumokokken: 1
Zalven en crèmes: 1
Behandel